# Real model

In [1]:
pip install ortools


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [3]:
repo_url = 'https://raw.githubusercontent.com/baertsch/MGT-530-SLO/main/'

In [4]:
vehicle_matrix = pd.read_excel(repo_url + 'vhc_matrix_city_excluded.xlsx')
vehicle_matrix = vehicle_matrix.iloc[0:, :1]
vehicle_matrix

,Places
0,16
1,91
2,50
3,20
4,127
5,100


In [5]:
v = vehicle_matrix.values.tolist()
v = [item[0] for item in v]

In [6]:
full_data = pd.read_csv(repo_url + 'full_data.csv')

In [7]:
d_venoge = full_data[['NPA', 'distance_to_venoge_km']]

## Wednesday

In [8]:
demand_wed = pd.read_csv(repo_url + 'Tickets_du_Mercredi.csv',header=None)

### Distance matrix

In [9]:
distance_matrix_wed = pd.read_csv(repo_url + 'data/dm_mercredi.csv',header=None)
distance_matrix_wed = distance_matrix_wed.iloc[1:,1:]

In [10]:
df = demand_wed.iloc[1:, 2:3]
df = df.astype('int64')
df = pd.merge(d_venoge, df, left_on='NPA', right_on=2)
df = df.iloc[:, 1:2]
depot_distances = df.iloc[:, 0].tolist()

# Insert depot as first column
distance_matrix_wed = distance_matrix_wed.copy()
distance_matrix_wed.insert(0, 'Depot', depot_distances)

# Insert depot as first row (must match new number of columns)
depot_row = [0.0] + depot_distances
distance_matrix_wed.loc[-1] = depot_row
distance_matrix_wed.index = distance_matrix_wed.index + 1
distance_matrix_wed = distance_matrix_wed.sort_index()

# Reset column names and index to integers
distance_matrix_wed.columns = range(distance_matrix_wed.shape[1])
distance_matrix_wed.index = range(distance_matrix_wed.shape[0])

In [11]:
dm_wed = distance_matrix_wed.values.tolist()
for i in range(len(dm_wed)):
    dm_wed[i][i] = 0
dm_wed = [[9999 if (isinstance(x, float) and np.isinf(x)) else x for x in row] for row in dm_wed]
dm_wed = [[int(float(x)) for x in row] for row in dm_wed]


### Demand matrix

In [12]:
demand_wed = demand_wed.iloc[1:, 4:]

In [13]:
d_wed = demand_wed.values.tolist()
d_wed = [int(float(item[0])) for item in d_wed]
d_wed = [0] + d_wed

### Vehicle Matrix

In [14]:
v_wed_full = v * len(dm_wed)

In [15]:
def create_data_model(subset_distance_matrix, subset_demands, capacities):
    """Stores the data for the problem."""
    data = {}
    # Data multiplied by a factor of 10 to avoid non-integer numbers
    data['distance_matrix'] = subset_distance_matrix
    data['demands'] = subset_demands
    data['vehicle_capacities'] = capacities
    data['num_vehicles'] = len(capacities)
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console, listing unused vehicles separately."""
    total_distance = 0
    total_load = 0
    unused_vehicles = []

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_load = 0
        capacity = data['vehicle_capacities'][vehicle_id]
        route_output = f"Route for vehicle {vehicle_id} (Capacity: {capacity}):\n"

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            demand = data['demands'][node_index]
            route_load += demand
            route_output += (
                f" {node_index} ->"# (Demand: {demand}, Current Load: {route_load}) ->"
            )
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        end_node = manager.IndexToNode(index)
        route_output += f" {end_node} (Load: {route_load})\n"

        if route_load == 0:
            unused_vehicles.append(vehicle_id)
            continue  # Skip printing empty routes

        route_output += f"Distance of the route: {route_distance / 10:.2f} km\n"
        route_output += f"Load of the route: {route_load}\n"
        print(route_output)

        total_distance += route_distance
        total_load += route_load

    print(f"Total distance of all routes: {total_distance / 10:.2f} km")
    print(f"Total load of all routes: {total_load}")
    
    if unused_vehicles:
        print(f"Unused vehicles: {unused_vehicles}")
        print(f"Number of unused vehicles: {len(unused_vehicles)}")


def main():
    """Solve the CVRP problem."""

    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix= dm_wed, subset_demands=d_wed, capacities=v_wed_full)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        print_solution(data, manager, routing, solution)


main()

Route for vehicle 1 (Capacity: 91):
 0 -> 115 -> 0 (Load: 77)
Distance of the route: 1.80 km
Load of the route: 77

Route for vehicle 1660 (Capacity: 127):
 0 -> 293 -> 302 -> 305 -> 304 -> 111 -> 0 (Load: 127)
Distance of the route: 11.00 km
Load of the route: 127

Route for vehicle 1661 (Capacity: 100):
 0 -> 12 -> 0 (Load: 100)
Distance of the route: 0.80 km
Load of the route: 100

Route for vehicle 1666 (Capacity: 127):
 0 -> 294 -> 300 -> 142 -> 0 (Load: 125)
Distance of the route: 12.40 km
Load of the route: 125

Route for vehicle 1667 (Capacity: 100):
 0 -> 76 -> 0 (Load: 100)
Distance of the route: 0.80 km
Load of the route: 100

Route for vehicle 1672 (Capacity: 127):
 0 -> 244 -> 314 -> 309 -> 145 -> 0 (Load: 127)
Distance of the route: 12.70 km
Load of the route: 127

Route for vehicle 1673 (Capacity: 100):
 0 -> 78 -> 0 (Load: 100)
Distance of the route: 0.80 km
Load of the route: 100

Route for vehicle 1678 (Capacity: 127):
 0 -> 286 -> 292 -> 285 -> 121 -> 0 (Load: 126)
D

## Thursday

In [16]:
demand_thur = pd.read_csv(repo_url + 'Tickets_du_Jeudi.csv',header=None)

### Distance Matrix

In [17]:
distance_matrix_thur = pd.read_csv(repo_url + 'data/dm_jeudi.csv',header=None)
distance_matrix_thur = distance_matrix_thur.iloc[1:,1:]

In [18]:
df = demand_thur.iloc[1:, 2:3]
df = df.astype('int64')
df = pd.merge(d_venoge, df, left_on='NPA', right_on=2)
df = df.iloc[:, 1:2]
depot_distances = df.iloc[:, 0].tolist()

# Insert depot as first column
distance_matrix_thur = distance_matrix_thur.copy()
distance_matrix_thur.insert(0, 'Depot', depot_distances)

# Insert depot as first row (must match new number of columns)
depot_row = [0.0] + depot_distances
distance_matrix_thur.loc[-1] = depot_row
distance_matrix_thur.index = distance_matrix_thur.index + 1
distance_matrix_thur = distance_matrix_thur.sort_index()

# Reset column names and index to integers
distance_matrix_thur.columns = range(distance_matrix_thur.shape[1])
distance_matrix_thur.index = range(distance_matrix_thur.shape[0])

In [19]:
dm_thur = distance_matrix_thur.values.tolist()
for i in range(len(dm_thur)):
    dm_thur[i][i] = 0
dm_thur = [[9999 if (isinstance(x, float) and np.isinf(x)) else x for x in row] for row in dm_thur]
dm_thur = [[int(float(x)) for x in row] for row in dm_thur]

### Demand Matrix

In [20]:
demand_thur = demand_thur.iloc[1:, 4:]

In [21]:
d_thur = demand_thur.values.tolist()
d_thur = [int(float(item[0])) for item in d_thur]
d_thur = [0] + d_thur

### Vehicule Matrix

In [22]:
v_thur_full = v * len(dm_thur)

### CVRP

In [23]:
def main():
    """Solve the CVRP problem."""

    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix= dm_thur, subset_demands=d_thur, capacities=v_thur_full)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        print_solution(data, manager, routing, solution)


main()

Route for vehicle 1 (Capacity: 91):
 0 -> 3 -> 7 -> 0 (Load: 88)
Distance of the route: 0.50 km
Load of the route: 88

Route for vehicle 4 (Capacity: 127):
 0 -> 119 -> 159 -> 173 -> 96 -> 0 (Load: 125)
Distance of the route: 7.10 km
Load of the route: 125

Route for vehicle 10 (Capacity: 127):
 0 -> 15 -> 0 (Load: 100)
Distance of the route: 0.80 km
Load of the route: 100

Route for vehicle 1468 (Capacity: 127):
 0 -> 253 -> 276 -> 273 -> 228 -> 226 -> 217 -> 92 -> 0 (Load: 127)
Distance of the route: 16.30 km
Load of the route: 127

Route for vehicle 1469 (Capacity: 100):
 0 -> 55 -> 0 (Load: 100)
Distance of the route: 0.80 km
Load of the route: 100

Route for vehicle 1474 (Capacity: 127):
 0 -> 274 -> 236 -> 4 -> 0 (Load: 127)
Distance of the route: 15.40 km
Load of the route: 127

Route for vehicle 1475 (Capacity: 100):
 0 -> 29 -> 0 (Load: 100)
Distance of the route: 0.80 km
Load of the route: 100

Route for vehicle 1480 (Capacity: 127):
 0 -> 82 -> 98 -> 93 -> 0 (Load: 123)
Dist

## Friday

In [24]:
demand_fri = pd.read_csv(repo_url + 'Tickets_du_Vendredi.csv',header=None)

### Distance Matrix

In [25]:
distance_matrix_fri = pd.read_csv(repo_url + 'data/dm_vendredi.csv',header=None)
distance_matrix_fri = distance_matrix_fri.iloc[1:,1:]

In [26]:
df = demand_fri.iloc[1:, 2:3]
df = df.astype('int64')
df = pd.merge(d_venoge, df, left_on='NPA', right_on=2)
df = df.iloc[:, 1:2]
depot_distances = df.iloc[:, 0].tolist()

# Insert depot as first column
distance_matrix_fri = distance_matrix_fri.copy()
distance_matrix_fri.insert(0, 'Depot', depot_distances)

# Insert depot as first row (must match new number of columns)
depot_row = [0.0] + depot_distances
distance_matrix_fri.loc[-1] = depot_row
distance_matrix_fri.index = distance_matrix_fri.index + 1
distance_matrix_fri = distance_matrix_fri.sort_index()

# Reset column names and index to integers
distance_matrix_fri.columns = range(distance_matrix_fri.shape[1])
distance_matrix_fri.index = range(distance_matrix_fri.shape[0])

In [27]:
dm_fri = distance_matrix_fri.values.tolist()
for i in range(len(dm_fri)):
    dm_fri[i][i] = 0
dm_fri = [[9999 if (isinstance(x, float) and np.isinf(x)) else x for x in row] for row in dm_fri]
dm_fri = [[int(float(x)) for x in row] for row in dm_fri]

## Demand Matrix

In [28]:
demand_fri = demand_fri.iloc[1:, 4:]

In [29]:
d_fri = demand_fri.values.tolist()
d_fri = [int(float(item[0])) for item in d_fri]
d_fri = [0] + d_fri

### Vehicule Matrix

In [30]:
v_fri_full = v * len(dm_fri)

### CVRP

In [31]:
def main():
    """Solve the CVRP problem."""

    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix= dm_fri, subset_demands=d_fri, capacities=v_fri_full)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        print_solution(data, manager, routing, solution)


main()

Route for vehicle 0 (Capacity: 16):
 0 -> 128 -> 0 (Load: 13)
Distance of the route: 5.00 km
Load of the route: 13

Route for vehicle 1 (Capacity: 91):
 0 -> 120 -> 144 -> 168 -> 194 -> 226 -> 235 -> 237 -> 233 -> 232 -> 224 -> 217 -> 219 -> 203 -> 167 -> 0 (Load: 91)
Distance of the route: 14.80 km
Load of the route: 91

Route for vehicle 1295 (Capacity: 100):
 0 -> 12 -> 0 (Load: 100)
Distance of the route: 0.80 km
Load of the route: 100

Route for vehicle 1300 (Capacity: 127):
 0 -> 239 -> 146 -> 53 -> 0 (Load: 127)
Distance of the route: 14.90 km
Load of the route: 127

Route for vehicle 1301 (Capacity: 100):
 0 -> 40 -> 0 (Load: 100)
Distance of the route: 0.80 km
Load of the route: 100

Route for vehicle 1306 (Capacity: 127):
 0 -> 179 -> 187 -> 54 -> 0 (Load: 124)
Distance of the route: 7.50 km
Load of the route: 124

Route for vehicle 1307 (Capacity: 100):
 0 -> 32 -> 0 (Load: 100)
Distance of the route: 0.80 km
Load of the route: 100

Route for vehicle 1312 (Capacity: 127):
 0

## Saturday

In [32]:
demand_sat = pd.read_csv(repo_url + 'Tickets_du_Samedi.csv',header=None)

### Distance Matrix

In [33]:
distance_matrix_sat = pd.read_csv(repo_url + 'data/dm_samedi.csv',header=None)
distance_matrix_sat = distance_matrix_sat.iloc[1:,1:]

In [34]:
df = demand_sat.iloc[1:, 2:3]
df = df.astype('int64')
df = pd.merge(d_venoge, df, left_on='NPA', right_on=2)
df = df.iloc[:, 1:2]
depot_distances = df.iloc[:, 0].tolist()

# Insert depot as first column
distance_matrix_sat = distance_matrix_sat.copy()
distance_matrix_sat.insert(0, 'Depot', depot_distances)

# Insert depot as first row (must match new number of columns)
depot_row = [0.0] + depot_distances
distance_matrix_sat.loc[-1] = depot_row
distance_matrix_sat.index = distance_matrix_sat.index + 1
distance_matrix_sat = distance_matrix_sat.sort_index()

# Reset column names and index to integers
distance_matrix_sat.columns = range(distance_matrix_sat.shape[1])
distance_matrix_sat.index = range(distance_matrix_sat.shape[0])

In [35]:
dm_sat = distance_matrix_sat.values.tolist()
for i in range(len(dm_sat)):
    dm_sat[i][i] = 0
dm_sat = [[9999 if (isinstance(x, float) and np.isinf(x)) else x for x in row] for row in dm_sat]
dm_sat = [[int(float(x)) for x in row] for row in dm_sat]

### Demand Matrix

In [36]:
demand_sat = demand_sat.iloc[1:, 4:]

In [37]:
d_sat = demand_sat.values.tolist()
d_sat = [int(float(item[0])) for item in d_sat]
d_sat = [0] + d_sat

### Vehicule Matrix

In [38]:
v_sat_full = v * len(dm_sat)

### CVRP

In [39]:
def main():
    """Solve the CVRP problem."""

    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix= dm_sat, subset_demands=d_sat, capacities=v_sat_full)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        print_solution(data, manager, routing, solution)


main()

Route for vehicle 1 (Capacity: 91):
 0 -> 98 -> 154 -> 147 -> 108 -> 0 (Load: 89)
Distance of the route: 5.80 km
Load of the route: 89

Route for vehicle 4 (Capacity: 127):
 0 -> 59 -> 126 -> 139 -> 158 -> 0 (Load: 124)
Distance of the route: 6.10 km
Load of the route: 124

Route for vehicle 5 (Capacity: 100):
 0 -> 49 -> 0 (Load: 100)
Distance of the route: 0.80 km
Load of the route: 100

Route for vehicle 1319 (Capacity: 100):
 0 -> 35 -> 0 (Load: 100)
Distance of the route: 0.80 km
Load of the route: 100

Route for vehicle 1324 (Capacity: 127):
 0 -> 194 -> 196 -> 187 -> 96 -> 0 (Load: 124)
Distance of the route: 7.90 km
Load of the route: 124

Route for vehicle 1325 (Capacity: 100):
 0 -> 15 -> 0 (Load: 100)
Distance of the route: 0.80 km
Load of the route: 100

Route for vehicle 1330 (Capacity: 127):
 0 -> 176 -> 237 -> 178 -> 93 -> 0 (Load: 127)
Distance of the route: 11.20 km
Load of the route: 127

Route for vehicle 1331 (Capacity: 100):
 0 -> 41 -> 0 (Load: 100)
Distance of th